In [1]:
# downloading the data
# !wget -P ./data https://pythonprogramming.net/static/downloads/machine-learning-data/crypto_data.zip

In [2]:
import os
import pandas as pd

In [3]:
# VARIABLES

# path
CWD = os.getcwd()
DATADIR = os.path.join(CWD, 'data', 'crypto_data')

# ml
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT = 'LTC-USD'

In [4]:
# if future price is greater than current prices, buy the asset, coded as 1
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [5]:
# name of the columns
names = ['time', 'low', 'high', 'open', 'close', 'volume']
# import the data with pandas
df = pd.read_csv(os.path.join(DATADIR, 'LTC-USD.csv'), names=names)
df.head()

,time,low,high,open,close,volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978


In [6]:
main_df = pd.DataFrame()

# file names
ratios = ['BCH-USD', 'BTC-USD', 'ETH-USD', 'LTC-USD']

# join all files into one big dataframe called main_df
for ratio in ratios:
    dataset = os.path.join(DATADIR, '{}.csv'.format(ratio))
    df = pd.read_csv(dataset, names=names)
    
    df = df.rename(columns={'close': '{}_close'.format(ratio), 'volume': '{}_volume'.format(ratio)})
    df = df.set_index('time')
    df = df.loc[:, ['{}_close'.format(ratio), '{}_volume'.format(ratio)]]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
        
# make future price by negative shifting the data by FUTURE_PERIOD_PREDICT
main_df.loc[:, 'future'] = main_df.loc[:, '{}_close'.format(RATIO_TO_PREDICT)].shift(-FUTURE_PERIOD_PREDICT)

# make target by using the classify function
main_df.loc[:, 'target'] = list(map(classify,
                                    main_df.loc[:, '{}_close'.format(RATIO_TO_PREDICT)],
                                    main_df.loc[:, 'future']))

In [7]:
# check if everything worked
main_df.loc[:, ['{}_close'.format(RATIO_TO_PREDICT), 'future', 'target']].head(10)

,LTC-USD_close,future,target
time,,,
1528968660,96.580002,96.500000,0
1528968720,96.660004,96.389999,0
1528968780,96.570000,96.519997,0
1528968840,96.500000,96.440002,0
1528968900,96.389999,96.470001,1
1528968960,96.519997,96.400002,0
1528969020,96.440002,96.400002,0
1528969080,96.470001,96.400002,0
1528969140,96.400002,96.400002,0
